In [4]:
import numpy as np
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
import nltk
from nltk import sent_tokenize
import spacy
import string
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import mysql.connector
import datetime
import time
from keras.models import load_model
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'spacy'

In [5]:
model = load_model('model.h5')

NameError: name 'load_model' is not defined

In [6]:
# Load the vectorizer
tokenizerN = 'tokenizer.pkl'
tokenizer = pickle.load(open(tokenizerN, 'rb'))
# Load the vectorizer
encoderN = 'encoder.pkl'
encoder = pickle.load(open(encoderN, 'rb'))

NameError: name 'pickle' is not defined

In [7]:
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

In [8]:
def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE

In [9]:
def predict(text, include_neutral=True):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=300)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}  

In [10]:
# Main loop
while True:
    delay = 2
    now = datetime.datetime.now()
    date_time = now.strftime("%D/%M/%Y %H:%M:%S")
    print(date_time)
    time.sleep(2)
    try:
        # Connect to the database
        mydb = mysql.connector.connect(
            host="localhost",
            port="3306",
            user="root",
            password="",
            database="emotions"
        )
        MyCursor1 = mydb.cursor()
        
        # Fetch records with status=0
        Sql_Query1 = "SELECT `id`, `content`, `result` FROM `posts` WHERE `status` = 0 ;"
        MyCursor1.execute(Sql_Query1)
        record1 = MyCursor1.fetchall()
        mydb.commit()
        MyCursor1.close()
        print("MyCursor1")
        
        if record1:
            for row in record1:
                rowID = row[0]
                rowID1 = str(rowID)
                print(rowID1)
                rowQuestion1 = row[1]
                
                if rowQuestion1:
                    msg_bullying = rowQuestion1.lower()
                    
                    # Apply vectorizer and predict
                    res_bullying = predict(msg_bullying)
                    result = res_bullying["label"]
                    print('res_bullying',result)                    
                    print("XXXXXXXXXXX")
                    
                    if res_bullying is not None:
                        # Update record with the predicted result and set status=1
                        MyCursor2 = mydb.cursor()
                        Sql_Query2 = f"UPDATE `posts` SET `status` = 1, `result`='{result}' WHERE `id`= {rowID1};"
                        MyCursor2.execute(Sql_Query2)
                        mydb.commit()
                        MyCursor2.close()
                        print("MyCursor2")
                    else:
                        print("No result")
                else:
                    print("Invalid input: rowQuestion1 is None")
        else:
            print("No records found.")
        
        mydb.close()
        print("End step")
    except Exception as e:
        print(e)

NameError: name 'datetime' is not defined